In [1]:
import pandas as pd
import os
from itertools import product
import numpy as np

In [2]:
orders = ['LO', 'NLO', 'N2LO', 'N3LO']
Lambdas = [450, 500]

high_density = True

# if high_density:
# #     bodies = ['NN+3N']
# #     xs = ['0.5']
#     bodies = ['NN-only', 'NN+3N']
#     xs = ['0.', '0.5']
# else:
#     bodies = ['NN-only', 'NN+3N']
#     xs = ['0.', '0.5']

bodies = ['NN-only', 'NN+3N']
xs = ['0.', '0.5']
xs_names = {'0.': 'PNM', '0.5': 'SNM'}
fits = {
    (450, 'LO'): [1],
    (500, 'LO'): [6],
    (450, 'NLO'): [2],
    (500, 'NLO'): [7],
    (450, 'N2LO'): [1, 2, 3],
    (500, 'N2LO'): [4, 5, 6],
    (450, 'N3LO'): [7, 8, 9],
    (500, 'N3LO'): [10, 11, 12],
}
data_dir = os.path.abspath('../raw_data/')


i = 0
dfs = []

for order, Lambda, body, x in product(orders, Lambdas, bodies, xs):
    x_name = xs_names[x]
    
    if body == 'NN-only' and not high_density:
        filename = os.path.join(data_dir, f'{body}/EOS_x_{x}_{order}_EM{Lambda}new.txt')
        i += 1
        df = pd.read_csv(filename, sep='\t', header=0)
        df.rename(columns={'# kf': 'kf', 'HF_NN': 'MBPT_HF', 'Scnd_NN': 'MBPT_2',
                           'Trd_NN_tot': 'MBPT_3', 'Fth_tot': 'MBPT_4'}, inplace=True)
        df['Lambda'] = Lambda
        df['OrderEFT'] = order
        df['Body'] = body
        df['x'] = float(x)
        df['fit'] = np.nan
        dfs.append(df)
    else:
        for fit in fits[Lambda, order]:
            if high_density:
                subdir = '' if body == 'NN+3N' else 'NN_only/'
                filename = f'NN+3N_high_density/{x_name}/{subdir}EOS_high_den_x_{x}_Ham_{fit}_{order}_EM{Lambda}new.txt'
            else:
                filename = f'{body}/EOS_x_{x}_Ham_{fit}_{order}_EM{Lambda}new.txt'

            filename = os.path.join(data_dir, filename)
            df = pd.read_csv(filename, sep='\t', skiprows=[0], names=['kf', 'n', 'Kin', 'MBPT_HF', 'MBPT_2', 'MBPT_3', 'MBPT_4', 'total'])
            
            
#             if high_density:
#                 filename = f'NN+3N_high_density/SNM/dat_high_den_Ham_{fit}_{order}_EM{Lambda}new.dat'
#             else:
#                 filename = f'{body}/EOS_x_{x}_Ham_{fit}_{order}_EM{Lambda}new.txt'
#             filename = os.path.join(data_dir, filename)
            i += 1
#             if high_density:
#                 df = pd.read_csv(
#                     filename, sep='\t', header=None,
#                     names=['kf', 'n', 'MBPT_4', '??', '???']
#                 )
#             else:
#                 df = pd.read_csv(filename, sep='\t', header=0)
#                 df.rename(columns={'# kf': 'kf', 'HF_tot': 'MBPT_HF', 'Scnd_tot': 'MBPT_2',
#                                    'Trd_tot': 'MBPT_3', 'Fth_tot': 'MBPT_4'}, inplace=True)
            df['Lambda'] = Lambda
            df['OrderEFT'] = order
            df['Body'] = body
            df['x'] = float(x)
            if order in ['LO', 'NLO'] or body == 'NN-only':
                df['fit'] = np.nan
            else:
                df['fit'] = fit
            dfs.append(df)
            
            if body == 'NN-only' or high_density:
                break  # Only one fit value is given for these
            
            # Copy data for easier parsing later on. Given LO and NLO the same fit #s at N2LO and N3LO.
            # if order in ['LO', 'NLO']:
            #     for other_fit in [*fits[Lambda, 'N2LO'], *fits[Lambda, 'N3LO']]:
            #         if other_fit != fit:
            #             df = df.copy() # just in case it would mess with stuff already in dfs
            #             df['fit'] = other_fit
            #             dfs.append(df)
            
    

print('Num files read:', i)
df = pd.concat(dfs, ignore_index=True)

# filename1 = '/Users/jordanmelendez/Dropbox/Research/Matter/Matter_Convergence_Analysis/data/NN-only/EOS_x_0._LO_EM500new.txt'
# filename2 = '/Users/jordanmelendez/Dropbox/Research/Matter/Matter_Convergence_Analysis/data/NN+3N/EOS_x_0._Ham_4_N2LO_EM500new.txt'
# df1 = pd.read_csv(filename1, sep='\t', header=0)
# df1.rename(columns={'# kf': 'kf', 'HF_NN': 'MBPT_HF', 'Scnd_NN': 'MBPT_2',
#                     'Trd_NN_tot': 'MBPT_3', 'Fth_tot': 'MBPT_4'}, inplace=True)
# df2 = pd.read_csv(filename2, sep='\t', header=0)
# df2.rename(columns={'# kf': 'kf', 'HF_tot': 'MBPT_HF', 'Scnd_tot': 'MBPT_2',
#                     'Trd_tot': 'MBPT_3', 'Fth_tot': 'MBPT_4'}, inplace=True)

Num files read: 32


In [3]:
df.dtypes

kf          float64
n           float64
Kin         float64
MBPT_HF     float64
MBPT_2      float64
MBPT_3      float64
MBPT_4      float64
total       float64
Lambda        int64
OrderEFT     object
Body         object
x           float64
fit         float64
dtype: object

In [4]:
df['fit'].unique()

array([nan,  1.,  4.,  7., 10.])

In [5]:
df[df['OrderEFT'] == 'LO']

,kf,n,Kin,MBPT_HF,MBPT_2,MBPT_3,MBPT_4,total,Lambda,OrderEFT,Body,x,fit
0,1.13972,0.05,16.16071,-5.56660,-1.93533,0.01619,-0.03168,8.64329,450,LO,NN-only,0.0,NaN
1,1.21113,0.06,18.24937,-6.53108,-2.06096,0.10349,-0.04271,9.71811,450,LO,NN-only,0.0,NaN
2,1.27499,0.07,20.22456,-7.47207,-2.14500,0.17957,-0.05451,10.73255,450,LO,NN-only,0.0,NaN
3,1.33302,0.08,22.10753,-8.39215,-2.19485,0.24304,-0.07415,11.68942,450,LO,NN-only,0.0,NaN
4,1.38640,0.09,23.91344,-9.29313,-2.21891,0.29680,-0.09426,12.60394,450,LO,NN-only,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,1.64376,0.30,33.61562,-31.34471,-15.54186,0.59316,1.10862,-11.56917,500,LO,NN+3N,0.5,NaN
236,1.66182,0.31,34.35855,-32.19647,-15.43194,0.60399,1.02151,-11.64436,500,LO,NN+3N,0.5,NaN
237,1.67950,0.32,35.09352,-33.04140,-15.31417,0.61204,0.92375,-11.72626,500,LO,NN+3N,0.5,NaN
238,1.69682,0.33,35.82088,-33.87963,-15.18736,0.61980,0.83670,-11.78961,500,LO,NN+3N,0.5,NaN


In [6]:

savename = 'all_matter_data.csv'
if high_density:
    savename = 'all_matter_data_high_density.csv'
df.to_csv(savename, index=False)